In [1]:
import csv
import os, os.path
import arcpy
import requests
import pandas as pd

In [2]:
### Get Seoul Data ###
def getSeoulData(startIndex, endIndex):
    seoulAPIKey = "427146617665737435366162794b54"
    seoulRestUrl = "http://openapi.seoul.go.kr:8088/"+str(seoulAPIKey)+"/json/IotVdata018/"+str(startIndex)+"/"+str(endIndex)+"/"
    print("Request: ", seoulRestUrl)
    response = requests.get(seoulRestUrl)
    df = pd.DataFrame(response.json())
    return(df["IotVdata018"]["row"])


In [3]:
### Make CSV file ###
def writeCsvFile(csv_path, df):

    # Write Rows
    # Row Info : ["기관명","송신 서버 번호","데이터 번호","모델명", "시리얼(forigen key)", "서버타입" ,"사이트명" ,"방문자수", "날짜", "등록일자"]
    if not os.path.exists(csv_path):
        csv_file = open(csv_path, 'w', newline='')
        csv_writer = csv.writer(csv_file)
        header=["Orag_NM","Transmit_Server_NO","DATA_NO","Model_NM", "Site", "Server_Type" ,"Site_NM" ,"Visit_Count", "Date", "Regist_Date"] 
        csv_writer.writerow(header)
        for data in df:
            csv_writer.writerow(data.values())
    else:
        csv_file = open(csv_path, 'a', newline='')
        csv_writer = csv.writer(csv_file)
        for data in df:
            csv_writer.writerow(data.values())
    csv_file.close()

In [44]:
# Configure Requrest URL
seoulStartIndex = 650000
seoulEndIndex = 1000000

# Csv File Path
csv_path = 'D:\Data\\SeoulIoTData.csv'

### Seoul CSV Main ###
try:
    # Get Data
    startIndex = seoulStartIndex
    endIndex = startIndex+999
    while endIndex < seoulEndIndex:
        seoulData = getSeoulData(startIndex, endIndex)
        # Make Csv File
        writeCsvFile(csv_path, seoulData)
        print("Seoul CSV File Completed: from ", startIndex, " to ", endIndex)
        # Calcaulate the Next Request Value
        startIndex+=1000
        endIndex=startIndex+999

    print("Csv File Location: ", csv_path)

except Exception as e:
    print("Get Seoul Csv File Error: ", e)

Request:  http://openapi.seoul.go.kr:8088/427146617665737435366162794b54/json/IotVdata018/650000/650999/
Seoul CSV File Completed: from  650000  to  650999
Request:  http://openapi.seoul.go.kr:8088/427146617665737435366162794b54/json/IotVdata018/651000/651999/
Seoul CSV File Completed: from  651000  to  651999
Request:  http://openapi.seoul.go.kr:8088/427146617665737435366162794b54/json/IotVdata018/652000/652999/
Seoul CSV File Completed: from  652000  to  652999
Request:  http://openapi.seoul.go.kr:8088/427146617665737435366162794b54/json/IotVdata018/653000/653999/
Seoul CSV File Completed: from  653000  to  653999
Request:  http://openapi.seoul.go.kr:8088/427146617665737435366162794b54/json/IotVdata018/654000/654999/
Seoul CSV File Completed: from  654000  to  654999
Request:  http://openapi.seoul.go.kr:8088/427146617665737435366162794b54/json/IotVdata018/655000/655999/
Seoul CSV File Completed: from  655000  to  655999
Request:  http://openapi.seoul.go.kr:8088/4271466176657374353661

In [46]:
### Delete Duplicates ###
csv_check = pd.read_csv(csv_path, encoding='euc-kr', sep=",")
csv_check_after = csv_check.drop_duplicates() #subset=['Site','Server_Type','Site_NM','Visit_Count','Date','Regist_Date']
csv_check_after.to_csv(csv_path, index=False, encoding='euc-kr')

,Orag_NM,Transmit_Server_NO,DATA_NO,Model_NM,Site,Server_Type,Site_NM,Visit_Count,Date,Regist_Date
0,서울시,32.0,1.0,SDOT001,3037,00Original,3037,137,202208010200,2022-08-01 02:05:08.0
1,서울시,32.0,1.0,SDOT001,4024,00Original,4024,1,202208010200,2022-08-01 02:05:05.0
2,서울시,32.0,1.0,SDOT001,3036,00Original,3036,213,202208010200,2022-08-01 02:05:04.0
3,서울시,32.0,1.0,SDOT001,3034,00Original,3034,338,202208010200,2022-08-01 02:05:02.0
4,서울시,32.0,1.0,SDOT001,3033,00Original,3033,147,202208010200,2022-08-01 02:05:02.0


In [47]:
### ArcGIS Pro ###
sensor_path = "C:\\Users\\esrikr\\Documents\\ArcGIS\\Projects\\LotteGRS\\LotteGRS.gdb\\PopSensorLocation"
output_path = "C:\\Users\\esrikr\\Documents\\ArcGIS\Projects\\LotteGRS\\LotteGRS.gdb\\PopSensorLocation_JoinFeatures"

if arcpy.Exists(output_path):
    arcpy.Delete_management(output_path)

arcpy.gapro.JoinFeatures(sensor_path, csv_path, output_path , "JOIN_ONE_TO_MANY", '', None, '', None, "Site Site", None, '', None)
arcpy.management.ConvertTimeField(output_path, "Date", "yyyyMMddHHmm", "Date_Converted", "DATE", '')

<Result 'C:\\Users\\esrikr\\Documents\\ArcGIS\\Projects\\LotteGRS\\LotteGRS.gdb\\PopSensorLocation_JoinFeatures_training'>

In [55]:
df_file_path = "D:\Data\LotteGRS\PopSensor_Training.csv"
df = pd.read_csv(df_file_path, sep=",")

In [57]:
y = df.Visit_Count


In [60]:
df.columns
features = ['Lat', 'Long', 'Date_Converted']

In [61]:
X = df[features]
X.describe()

,Lat,Long
count,248965.000000,248965.000000
mean,37.550525,126.987935
std,0.049773,0.091236
min,37.467564,126.813337
25%,37.506233,126.908102
50%,37.550545,127.007188
75%,37.578786,127.043786
max,37.640683,127.143018


In [62]:
X.head()

,Lat,Long,Date_Converted
0,37.59116,126.943006,2022-09-06 18:50
1,37.59116,126.943006,2022-09-06 18:40
2,37.59116,126.943006,2022-09-06 18:30
3,37.59116,126.943006,2022-09-06 18:20
4,37.59116,126.943006,2022-09-06 18:10


In [ ]:
from sklearn.tree import DecisionTreeRegressor